In [52]:
import streamlit as st
import numpy as np
import pandas as pd
from PIL import Image
import gspread
import json
import pandas as pd
#ServiceAccountCredentials：Googleの各サービスへアクセスできるservice変数を生成します。
from oauth2client.service_account import ServiceAccountCredentials

#2つのAPIを記述しないとリフレッシュトークンを3600秒毎に発行し続けなければならない
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
json_path = 'kitaichi-fd2dd716665b.json'
#認証情報設定
#ダウンロードしたjsonファイル名をクレデンシャル変数に設定（秘密鍵、Pythonファイルから読み込みしやすい位置に置く）
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_path, scope)
#OAuth2の資格情報を使用してGoogle APIにログインします。
gc = gspread.authorize(credentials)
# １．ファイル名を指定してワークブックを選択
workbook = gc.open('収支表')
sh_shop = workbook.worksheet('店一覧')
sh_slot_kisyu = workbook.worksheet('スロット機種一覧')
df_shop = pd.DataFrame(sh_shop.get_all_values()[1:], columns=sh_shop.get_all_values()[0])
df_slot_kisyu = pd.DataFrame(sh_slot_kisyu.get_all_values()[1:], columns=sh_slot_kisyu.get_all_values()[0])
sh = workbook.worksheet('収支')
df = pd.DataFrame(sh.get_all_values()[1:], columns=sh.get_all_values()[0])

In [46]:
list(df['期待値'])

['2700', '-305', '900', '1000', '400', '900', '600']

In [48]:
sum([int(i) for i in list(df['期待値'])])

6195

In [49]:
sum([int(i) for i in list(df['収支金額'])])

3870

In [57]:
df[df['店舗名'] == 'ガイア小郡']['期待値']

0    2700
Name: 期待値, dtype: object

In [53]:
df

,年月日,開始時間,終了時間,店舗名,台番号,機種名,開始G数,当選G数,ヤメG数,投資金額,投資メダル枚数,回収金額,回収メダル枚数,期待値,収支金額,収支メダル枚数,備考
0,2022-12-02,10:54:38,15:36:50,ガイア小郡,1300,ヴァルヴレイヴ,856,1400,0,20680,1034,25000,1300,2700,4320,266,
1,2022-12-05,16:28:17,16:29:09,ダイナム宇部港町店,130,ブラックラグーン,0,387,0,4500,450,1110,111,-305,-3390,-339,
2,2022-12-05,16:32:34,16:33:10,ダイナム宇部港町店,61,花火絶景,351,457,0,1500,150,760,76,900,-740,-74,
3,2022-12-05,16:33:59,16:34:51,ダイナム宇部港町店,62,花火絶景,431,513,0,1500,150,4120,412,1000,2620,262,
4,2022-12-06,18:32:06,18:52:01,ダイナム宇部港町店,71,ガメラ,331,389,0,1000,100,6020,602,400,5020,502,
5,2022-12-09,20:53:05,21:10:58,ダイナム宇部港町店,176,ニャル子さん,142,227,0,1500,150,780,78,900,-720,-72,
6,2022-12-09,21:13:34,21:36:03,ダイナム宇部港町店,97,番長ゼロ,359,592,0,4000,400,760,76,600,-3240,-324,


In [93]:
days_list=[]
day_list = list(set(df['年月日']))
for day in day_list:
    day_df = df[df['年月日']==day]
    sum([int(i) for i in list(day_df['期待値'])])
    sum([int(i) for i in list(day_df['収支金額'])])
    days_list.append([day,sum([int(i) for i in list(day_df['期待値'])]),sum([int(i) for i in list(day_df['収支金額'])])])
day_df = pd.DataFrame(days_list,columns=['年月日','期待値','収支金額']).sort_values('年月日').reset_index(drop=True)
suii_list = []
kitaiti_sum = 0
syushi_sum = 0
for i in range(len(list(day_df['期待値']))):
    if i == 0:
        kitaiti_sum = list(day_df['期待値'])[i]
        syushi_sum = list(day_df['収支金額'])[i]
    else:
        kitaiti_sum = list(day_df['期待値'])[i] + kitaiti_sum
        syushi_sum = list(day_df['収支金額'])[i] + syushi_sum
    
    suii_list.append([list(day_df['年月日'])[i],kitaiti_sum,syushi_sum])
suii_df = pd.DataFrame(suii_list,columns=['年月日','期待値','収支金額'])

['2022-12-05', '2022-12-09', '2022-12-02', '2022-12-06']


In [94]:
suii_df

,年月日,期待値,収支金額
0,2022-12-02,2700,4320
1,2022-12-05,4295,2810
2,2022-12-06,4695,7830
3,2022-12-09,6195,3870
